In [ ]:
#import torch
#from transformers import pipeline



In [ ]:
import pandas as pd

final_data = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/annotation_data_translated_with_blame.csv")


#df_trans.pop('Unnamed: 0')

final_data.head()



,Unnamed: 0,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,translated_text,blame_in_text
0,0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK,"The meeting is open. Under the Constitution, t...","[{'sequence': 'The meeting is open.', 'labels'..."
1,1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK,I would like to thank Things for the confidenc...,[{'sequence': 'I would like to thank Things fo...
2,2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK,"Twenty-five years ago, a majority of the peopl...","[{'sequence': 'Twenty-five years ago, a majori..."
3,3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK,From the Committee to the Committee of the Reg...,"[{'sequence': ""From the Committee to the Commi..."
4,4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK,We are faced with a very crucial parliamentary...,[{'sequence': 'We are faced with a very crucia...


In [3]:
test_data = final_data.loc[0:4]



In [47]:
single_list

'[{\'sequence\': \'I would like to thank Things for the confidence they have shown me by re-electing me as President.\', \'labels\': [\'praise\', \'neutral\', \'blame\'], \'scores\': [0.9563701152801514, 8.46099283080548e-07, 1.2345350342002348e-07]}, {\'sequence\': \'I shall make every effort, in close cooperation with my colleagues in the Bureau, to fulfil my duties as President of the Things.\', \'labels\': [\'neutral\', \'praise\', \'blame\'], \'scores\': [0.9998986124992371, 5.008435891795671e-06, 1.2553725525776827e-07]}, {\'sequence\': \'I welcome our guests today.\', \'labels\': [\'neutral\', \'blame\', \'praise\'], \'scores\': [0.9999908208847046, 2.363743078603875e-06, 1.7596008206055558e-07]}, {\'sequence\': \'I warmly welcome her Majesty the Queen, to His Royal Highness the Prince, to His Royal Highness Crown Prince Frederik, to His Royal Highness Prince Joachim, to Her Royal Highness Princess Alexandra and to Her Royal Highness Princess Benedikte.\', \'labels\': [\'praise\

In [ ]:
'''def extract_blame_from_paragraph(input):

        sentence_list = ast.literal_eval(input)

        blame_binary = []

        for sent_dict in sentence_list:
                label_scores = dict(zip(sent_dict['labels'], sent_dict['scores']))
                blame_probs = (label_scores.get('blame', 0.0))
                praise_probs = (label_scores.get('praise', 0.0))
                neutral_probs = (label_scores.get('neutral', 0.0))

                if (blame_probs >= praise_probs) & (blame_probs >= neutral_probs) & (blame_probs >= 0.80):
                        blame = 1
                else:
                        blame = 0
                
                blame_binary.append(blame)

        return blame_binary

final_data['blame_binary'] = final_data['blame_in_text'].apply(extract_blame_from_paragraph)'''

In [2]:
import ast
import swifter  # optional for parallel CPU usage

def extract_blame_from_paragraph_lookup(input_str):
    """
    Returns a binary list for blame per sentence:
    1 if blame is highest among labels and >= 0.8, else 0
    Handles arbitrary label order.
    """
    try:
        
        sentence_list = ast.literal_eval(input_str)
    except Exception:
        return []

    # List comprehension is faster than appending in a loop
    blame_binary = [
        int(
            (label_score := {label: score for label, score in zip(sent['labels'], sent['scores'])})['blame']
            >= max(label_score.get('praise', 0.0), label_score.get('neutral', 0.0), 0.8)
        )
        for sent in sentence_list
    ]

    return blame_binary


In [3]:
import ipywidgets

# Parallelize using swifter
final_data['blame_binary'] = final_data['blame_in_text'].swifter.apply(extract_blame_from_paragraph_lookup)




Pandas Apply:   0%|          | 0/36314 [00:00<?, ?it/s]

In [9]:
final_data.pop('Unnamed: 0')
final_data.to_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/annotation_data_translated_with_blame_heuristics.csv", index = False)

In [10]:
final_data.head()

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,translated_text,blame_in_text,blame_binary
0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK,"The meeting is open. Under the Constitution, t...","[{'sequence': 'The meeting is open.', 'labels'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK,I would like to thank Things for the confidenc...,[{'sequence': 'I would like to thank Things fo...,"[0, 0, 0, 0, 0, 0, 0, 0]"
2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK,"Twenty-five years ago, a majority of the peopl...","[{'sequence': 'Twenty-five years ago, a majori...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK,From the Committee to the Committee of the Reg...,"[{'sequence': ""From the Committee to the Commi...","[0, 0]"
4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK,We are faced with a very crucial parliamentary...,[{'sequence': 'We are faced with a very crucia...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
'''OBS: can quite easily (maybe) be made into a multiclassification model 
(blame/praise/neurtral). 
But due to timely constraints, the scope has been limited for this paper'''

In [2]:
import pandas as pd
final_data = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/annotation_data_translated_with_blame_heuristics.csv")

In [3]:
final_data.head()

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,translated_text,blame_in_text,blame_binary
0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK,"The meeting is open. Under the Constitution, t...","[{'sequence': 'The meeting is open.', 'labels'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK,I would like to thank Things for the confidenc...,[{'sequence': 'I would like to thank Things fo...,"[0, 0, 0, 0, 0, 0, 0, 0]"
2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK,"Twenty-five years ago, a majority of the peopl...","[{'sequence': 'Twenty-five years ago, a majori...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK,From the Committee to the Committee of the Reg...,"[{'sequence': ""From the Committee to the Commi...","[0, 0]"
4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK,We are faced with a very crucial parliamentary...,[{'sequence': 'We are faced with a very crucia...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
import ast

def get_rows_with_blame(df, col="blame_binary"):
    """
    Returns row indices where the list in `col` contains at least one 1.
    Handles both real lists and stringified lists.
    """
    indices = []
    for i, values in zip(df.index, df[col]):
        if isinstance(values, str):  # convert only if it's a string
            values = ast.literal_eval(values)
        if 1 in values:
            indices.append(i)
    return indices


def get_rows_and_positions(df, col="blame_binary"):
    """
    Returns {row_index: [positions_of_1s]}.
    Handles both real lists and stringified lists.
    """
    results = {}
    for i, values in zip(df.index, df[col]):
        if isinstance(values, str):
            values = ast.literal_eval(values)
        ones = [j for j, v in enumerate(values) if v == 1]
        if ones:
            results[i] = ones
    return results


In [9]:
row_indices = get_rows_with_blame(final_data, col="blame_binary")
print(row_indices[:10])

rows_with_positions = get_rows_and_positions(final_data, col="blame_binary")
print(rows_with_positions)


[2, 4, 7, 17, 18, 22, 32, 35, 41, 42]
{2: [230], 4: [88], 7: [3, 4, 5], 17: [7], 18: [0, 8], 22: [0], 32: [2], 35: [0], 41: [3], 42: [13], 52: [1], 57: [15, 82], 62: [0], 67: [1, 2], 68: [7], 80: [1], 88: [0], 91: [4, 5], 93: [6], 94: [5, 6], 99: [2], 101: [1], 103: [6, 61], 106: [3], 109: [5, 8], 122: [2], 128: [14], 138: [1, 5], 140: [2], 143: [10], 145: [60, 61, 68, 98], 153: [1, 3], 154: [4, 7], 156: [2], 162: [2, 4, 8, 78, 97, 112, 123, 125, 129, 130, 131], 169: [4, 6], 171: [3, 4, 7], 172: [2], 173: [5, 17], 175: [24, 27, 33, 52, 61, 70, 78, 95, 123], 179: [43, 65, 67, 73, 81], 181: [14, 24], 183: [93], 191: [9], 193: [6], 197: [7], 199: [4], 200: [17], 203: [0], 205: [8, 13], 209: [6, 10], 216: [82], 229: [6], 247: [1], 253: [4], 261: [9], 273: [34], 274: [21], 278: [0], 285: [3], 300: [14, 16], 313: [0], 320: [3, 7, 8], 325: [11], 327: [1, 7, 11], 332: [1], 347: [9], 353: [11], 407: [4], 408: [1], 455: [2], 458: [2], 496: [2], 500: [3, 4], 504: [3], 511: [4], 531: [0], 534: [5]

In [14]:
import ast

def extract_blamed_sentences(df, text_col="blame_in_text", blame_col="blame_binary"):
    """
    Extracts the sentences (sequences) from `text_col` where `blame_col` has 1.
    
    Assumes:
      - `text_col` is a stringified list of dicts with 'sequence', 'labels', 'scores'
      - `blame_col` is either a list of 0/1 or a stringified list
    """
    results = {}

    for idx, (text_entry, blame_entry) in enumerate(zip(df[text_col], df[blame_col]), start=0):
        # parse both columns if they are strings
        if isinstance(text_entry, str):
            text_entry = ast.literal_eval(text_entry)
        if isinstance(blame_entry, str):
            blame_entry = ast.literal_eval(blame_entry)

        blamed_sentences = [
            sent_dict["sequence"] 
            for sent_dict, flag in zip(text_entry, blame_entry) 
            if flag == 1
        ]

        if blamed_sentences:
            results[idx] = blamed_sentences
    
    return results


In [15]:
blamed = extract_blamed_sentences(final_data, "blame_in_text", "blame_binary")
print(blamed[0])   # → list of sentences marked as blame in first row


KeyError: 0

In [16]:
blamed

{2: ['The government will also continue its offensive environmental policy.'],
 4: ["In stark contrast to that view, the reaction of the agricultural organisations and the Left and the Conservatives' financial legislation are chemically cleared of taxes as a control tool, i.e. indifference to the aquatic environment to satisfy the agricultural lobby's hawks."],
 7: ['The way the debate had taken place, Mr Torben Lund said, it was low and demeaning and embarrassing.',
  'Allow me to say that it is low and demeaning and embarrassing of the current government that it has not taken up the debate until now when there is a parliamentary election and a municipal election in sight.',
  'I find that quite embarrassing.'],
 17: ["That's how it's socially well screwed up."],
 18: ["It is truly deeply shameful that the Social Democrat rapporteur can try to explain away the international comparisons which have been made by school pupils' knowledge of reading, arithmetic, physics and chemistry.",
  

In [ ]:
#Blame and praise
# entailsments
#does not entail an actually entailment

#Blame vs endorsement

In [ ]:
#Also report initial inspection of model wher praise and neutral was added to the hypothesis paramenter instead of blame/not blame.
# something about hieracivcal order and the words not being complete opisats and therefore the relative probabilities entails needed information
# in addition to the absolute probability